## **Clonando e instalando o Tensorflow API Object Detection** 

Para usar a API do Tensorflow para detecção de objetos é necessário clonar seu repositório no Github.

In [ ]:
!git clone https://github.com/tensorflow/models.git

A API do Tensorflow depende dos chamados `protocol buffers`, também conhecidos como `protobufs`.<br/>
Abaixo estamos compilando todos os `protobufs` na pasta `object_detection/protos` em Python.

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

Instalando a API do Tensorflow

In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Executando o teste de construtor de modelo

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## **Configurações Google Cloud Service Bucket**

Setando várias para facilitar o acesso

In [5]:
PROJECT = "Your Project Here"
YOUR_GCS_BUCKET = "Your GCS Bucket Here"

In [ ]:
!gcloud config set project '{PROJECT}'

Cria acesso ao bucket que se deseja acessar

In [ ]:
!gsutil mb gs://'{YOUR_GCS_BUCKET}'

Autenticação ao Bucket

In [ ]:
!gcloud auth login

In [ ]:
!gcloud auth application-default login

## **Definindo os variaveis necessarias**

In [10]:
model_dir = "gs://"+YOUR_GCS_BUCKET+"/train"
labelmap_path = "gs://"+YOUR_GCS_BUCKET+"/data/object-detection.pbtxt"

pipeline_config_path = "gs://"+YOUR_GCS_BUCKET+"/data/mobilenet_v2.config"
fine_tune_checkpoint = "gs://"+YOUR_GCS_BUCKET+"/train/your checkpoint here"

## **Exporta o modelo treinado**

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {pipeline_config_path} \
    --trained_checkpoint_dir {model_dir} \
    --output_directory /content/export_model/

## **Imports**

In [12]:
import numpy as np
import os
import tensorflow as tf

from PIL import Image
from IPython.display import display
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## **Funções auxiliares**

In [13]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)

  # A entrada do modelo deve ser um tensor
  input_tensor = tf.convert_to_tensor(image)

  # O modelo espera um lote de imagens
  # Para isso adicione um eixo com 'tf.newaxis'
  input_tensor = input_tensor[tf.newaxis,...]

  model = model.signatures['serving_default']

  # Inferencia
  output_dict = model(input_tensor)

  # Todas as saídas são tensores. 
  # Converta em matrizes numpy e pegue o índice [0] para remover a dimensão 
  # Estamos interessados ​​apenas nas primeiras num_detecções
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() for key, value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # As classes devem ser inteiros
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  return output_dict

Exibe a imagem com a inferência realizada

In [14]:
def show_inference(model, image_path, category):
  # A representação da imagem é baseada em array e será usada posteriormente para 
  # preparar a imagem resultante com bounding box

  image_np = np.array(Image.open(image_path))
  
  # Realiza detecção
  output_dict = run_inference_for_single_image(model, image_np)

  # Retorna imagem com resultado da inferencia
  vis_util.visualize_boxes_and_labels_on_image_array(image_np, 
                                                     output_dict['detection_boxes'], 
                                                     output_dict['detection_classes'],
                                                     output_dict['detection_scores'], 
                                                     category, 
                                                     instance_masks = output_dict.get('detection_masks_reframed', None),
                                                     use_normalized_coordinates = True,
                                                     line_thickness=8)
  
  display(Image.fromarray(image_np))

Carrega o modelo

In [15]:
def load_model(model_name):
  model_dir = model_name+"/saved_model"
  model = tf.saved_model.load(str(model_dir))
  return model

Converte imagens para o formato jpg

In [16]:
def load_img(img_file):
  im = Image.open(img_file)
  img = im.convert("RGB")
  img.save(img_file)
  return img_file

Transforma o labelmap em um vetor que utiliza chaves para buscar itens

In [17]:
def transform_labelmap():
  
  # Transforma o labelmap em um vetor de objetos com as categorias
  category_index = label_map_util.create_categories_from_labelmap(labelmap_path, use_display_name=True)
  category={}

  # Associa o ID a chave do objeto
  for ci in category_index:
      category[ci['id']]={}
      for k in ci.keys():
          if k =='id': continue
          category[ci['id']][k]=ci[k]

  return category

## **Inferências**

In [18]:
model_name = '/content/export_model'
model = load_model(model_name)

Realiza inferência em uma imagem de entrada

In [ ]:
show_inference(model,load_img("/content/dog.jpg"),transform_labelmap())